In [3]:
import csv
import json
from datetime import timedelta
from urllib.request import urlopen
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go


In [ ]:
print("Wplyw covid na swiat")

In [ ]:
#1.Porównywanie czy wraz ze wzrostem zachorowań wzrasta równiez ogólny poziom biedy
#2.Czy za każdym jak był wzrost zachorowań  to ceny kryptowalut się gwałtowanie zmieniały(procentowo)
#3.Jak ilość śmierci w 5 największych państwach wpłyneło na średnią pensję informatyków w pythonie w polsce
#4.Czy w tych państwach w których był największy covid podskoczyła liczba samobójstw i liczbę stwierdzonych depresji?
#5.Czy wielkość kraju miała wpływ na szybkość(rekord szybkości rozprzestrzeniania się )
#6.Jak poziom szowinizmu i
#7. Czy w krajach w których jest starsze społeczeńśtwo i jest więcej szowinistów było więcej śmierci wśród męzczyzn
#8.Czy w krajach w których średni poziom inteligencji jest najwyższy było najwięcej szczepionek

In [ ]:
#import danych
full_table = pd.read_csv('Dataset-recent.csv')
bitCoinPrice= pd.read_csv('coin_BitCoin.csv')
genderData=pd.read_csv('summary_data_clean.csv')
